# ナレッジソースの作成とナレッジエージェントの実行

Azure AI Search の新機能 Agentic retrieval では、新しくナレッジソースを追加できるようになり、LLM が検索対象のインデックスを自律的に選べるようになりました。さらに、LLM を活用した回答合成機能も利用できるようになっています。これにより、エージェントは複数のインデックスにまたがって情報をグラウンディングし、引用付きの回答を生成できるようになりました。

https://learn.microsoft.com/ja-jp/azure/search/search-knowledge-source-overview

In [ ]:
!pip install azure-search-documents==11.7.0b1

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# 環境変数から設定を取得
AZURE_OPENAI_ENDPOINT= os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY= os.getenv("AZURE_OPENAI_API_KEY")

SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
# OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
# OPENAI_KEY = os.getenv("OPENAI_KEY")

In [ ]:
search_api_version = "2025-08-01-preview"  # 11.7.0b1で使用される新しいAPIバージョン

# SearchIndexClientの初期化
search_client = SearchIndexClient(
    endpoint=SEARCH_ENDPOINT,
    credential=AzureKeyCredential(SEARCH_KEY),
    api_version=search_api_version
)

## Agentic retrieval の前提条件

https://learn.microsoft.com/ja-jp/azure/search/search-agentic-retrieval-how-to-index#criteria-for-agentic-retrieval

上記を参照して任意のインデックスを作成してください。

### 参考データセット: busho-index
https://github.com/nohanaga/busho-index

戦国武将の Wikipedia ページ

## 1. Create a knowledge source
この手順では、以前に作成したインデックスを対象とするナレッジソースを作成します。次の手順では、ナレッジソースを使用してエージェント型検索を調整するナレッジエージェントを作成します。

ナレッジ ソースは、エージェント検索用の追加のプロパティで検索インデックスをラップします。 ナレッジ エージェントで必要な定義です。 特定のナレッジ ソースを作成する方法に関するガイダンスを提供しますが、通常は次のことができます。

- 検索サービスで最上位レベルのリソースとして複数のナレッジ ソースを作成します。

- ナレッジ エージェント内の 1 つ以上のナレッジ ソースを参照します。 エージェント検索パイプラインでは、1 つの要求で複数のナレッジ ソースに対してクエリを実行できます。 サブクエリは、ナレッジ ソースごとに生成されます。 上位の結果が取得応答で返されます。

    - `create_knowledge_source`: すでに存在している場合、`DuplicateKnowledgeSource` Error が発生します。
    - `create_or_update_knowledge_source`: 新しいナレッジソースを作成するか、既存のナレッジソースを更新します。


### ナレッジ ソースに関する重要なポイント
- 作成パス: 最初にナレッジ ソースを作成してから、ナレッジ エージェントを作成します。 削除パス: ナレッジ エージェントを更新または削除し、ナレッジ ソースを最後に削除します。

- ナレッジ ソース、そのインデックス、ナレッジ エージェントはすべて、同じ検索サービス上に存在する必要があります。

- 各ナレッジ ソースは正確に 1 つのインデックスを指し、そのインデックスは エージェント検索の条件を満たす必要があります。

- ナレッジ ソースごとに、ナレッジ エージェントはクエリ実行用の追加のプロパティを提供します。 `KnowledgeSourceReference` プロパティは 、クエリの計画に影響します。 `KnowledgeAgentOutputConfiguration` プロパティは、クエリ出力に影響します。




https://learn.microsoft.com/ja-jp/azure/search/search-knowledge-source-overview

In [ ]:
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters

index_name = "Your-index"  # 既存のインデックス名を指定
knowledge_source_name = "wikipedia-knowledge-source"

# Search Indexを知識ソースとして使用
search_index_params = SearchIndexKnowledgeSourceParameters(
    search_index_name=index_name  # 既存のインデックス名を指定,必須
    # source_data_select="id,page_chunk,page_number", #参照されるソースデータの追加フィールドを要求するために使用されます。
)

knowledge_source = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    search_index_parameters=search_index_params, #知識ソースのパラメータ。必須。
    description="Wikipedia のナレッジソース"
)

try:
    # Knowledge Source の作成・アップデート
    search_client.create_or_update_knowledge_source(knowledge_source=knowledge_source, api_version=search_api_version)
    print(f"Knowledge Source '{knowledge_source.name}' を作成しました")

    # Knowledge Source の作成のみ
    # created_source = search_client.create_knowledge_source(knowledge_source)
    # print(f"Knowledge Source '{created_source.name}' を作成しました")

except Exception as e:
    print(f"Knowledge Source の作成中にエラーが発生しました: {e}")
    print(f"エラーの種類: {type(e).__name__}")


### Get Knowledge Source

In [ ]:
# Knowledge Sourceの取得
retrieved_source = search_client.get_knowledge_source(knowledge_source_name)
print(f"取得したKnowledge Source: {retrieved_source.description}")
print(retrieved_source)

### List Knowledge Source

In [ ]:
# Knowledge Sourceの一覧取得
sources = search_client.list_knowledge_sources()
sources_list = list(sources)
print(f"見つかったKnowledge Source数: {len(sources_list)}")

for source in sources_list:
    print(f"- {source.name}: {source.description}")

### Delete Knowledge Source
Knowledge Source が Knowledge Agent にリンクされている場合、先に Knowledge Agent を削除する必要がある。

In [ ]:
# response = search_client.delete_knowledge_source(knowledge_source_name)
# print(response)

## 2. Create a knowledge agent
このステップでは、ナレッジソースとLLMデプロイメントをラッパーとして扱うナレッジエージェントを作成します。

Azure AI Search では、 ナレッジエージェントは、Agentic retrieval ワークロードで使用するチャット完了モデルへの接続を表す最上位レベルのリソースです。 ナレッジエージェントは、LLM を利用した情報取得パイプラインの `retrieve` メソッド によって使用されます。

ナレッジ エージェントによって次のものが指定されます。

- 検索可能なコンテンツを指すナレッジ ソース (1 つ以上)
- クエリの計画と回答の定式化のための推論機能を提供するチャット完了モデル
- パフォーマンス最適化のプロパティ (クエリ処理時間の制約)

ナレッジ エージェントを作成した後は、いつでもそのプロパティを更新できます。ナレッジエージェントが使用中の場合、更新は次のジョブで有効になります。

In [ ]:
from azure.search.documents.indexes.models import (
    KnowledgeAgent,
    KnowledgeAgentAzureOpenAIModel,
    KnowledgeSourceReference,
    AzureOpenAIVectorizerParameters,
    KnowledgeAgentOutputConfiguration,
    KnowledgeAgentOutputConfigurationModality,
    KnowledgeAgentRequestLimits
)

In [ ]:
knowledge_agent_name = "wikipedia-agent"

# 正しいAzure OpenAIモデルの設定
openai_params = AzureOpenAIVectorizerParameters(
    resource_url=AZURE_OPENAI_ENDPOINT,
    deployment_name="gpt-4.1-mini",            # サポートされているモデル名に変更
    api_key=AZURE_OPENAI_API_KEY,        
    model_name="gpt-4.1-mini"
)

# Knowledge Agentモデルの設定
agent_model = KnowledgeAgentAzureOpenAIModel(
    azure_open_ai_parameters=openai_params
)

# Knowledge Sourceの参照
knowledge_source_ref = KnowledgeSourceReference(
    name=knowledge_source_name,
    # reranker_threshold=2.5, #これをセットするとmodelQueryPlanningが走ります
    # always_query_source=True,
    # include_references=False,
    # include_reference_source_data=False,
    # max_sub_queries=3
)

# 出力設定
# Answer synthesis (preview)
output_config = KnowledgeAgentOutputConfiguration(
    modality=KnowledgeAgentOutputConfigurationModality.ANSWER_SYNTHESIS,
    # modality=KnowledgeAgentOutputConfigurationModality.EXTRACTIVE_DATA,
    include_activity=True, # 検索結果にアクティビティ情報を含めるべきかを示します
    attempt_fast_path=False, # 高速パスを試みる。エージェントがモデル呼び出しをバイパスし、最新のチャットメッセージをナレッジソースへの直接クエリとして発行を試みるべきかを示します。
    # answer_instructions="" #ナレッジエージェントが回答生成時に考慮する指示。
)

# リクエスト制限設定
request_limits = KnowledgeAgentRequestLimits(
    max_runtime_in_seconds=30,
    max_output_size=5000 
)

# Knowledge Agentの作成
agent = KnowledgeAgent(
    name=knowledge_agent_name, 
    models=[agent_model],
    knowledge_sources=[ knowledge_source_ref], #複数指定可
    output_configuration=output_config,
    request_limits=request_limits,
    retrieval_instructions="質問に関する情報を検索し、適切な回答を提供してください。",
    description="このナレッジエージェントは、関連性のない複数のサンプルインデックスに向けられた質問を処理します。"
)

# Knowledge Agentの作成・アップデート
try:
    created_agent = search_client.create_or_update_agent(agent)
    print(f"Knowledge Agent '{created_agent.name}' を作成しました")
except Exception as e:
    print(f"エラー: {e}")

## 3. 新プロパティ解説
今回複数の新プロパティが追加されました。**各プロパティはセットするクラスが異なるため注意**してください。

### 3.1. Knowledge agents
ナレッジエージェント定義のアーキテクチャ変更により、`targetIndexes` の代わりに1つ以上の `knowledge_sources` が必要となり、`defaultMaxDocsForReranker` は非推奨となりました。新しい `retrieval_instructions` および `output_configuration` プロパティにより、クエリ計画と実行の制御性が向上しました。

https://learn.microsoft.com/ja-jp/azure/search/search-agentic-retrieval-how-to-create

### 3.2. Answer synthesis（回答合成）
既定では、Azure AI Search の ナレッジエージェントがデータ抽出を実行し、ナレッジソースから生のグラウンド チャンクを返します。データ抽出は特定の情報を取得するのに役立ちますが、複雑なクエリに必要なコンテキストと推論がありませんでした。

エージェントを構成して 回答合成を実行できます。この合成では、**デプロイされたチャット完了モデルを使用して自然言語でクエリに回答**します。各回答には、取得したソースへの引用が含まれており、箇条書きの使用など、指定した指示に従います。

- `ANSWER_SYNTHESIS`: LLM が応答用の回答を生成する
- `EXTRACTIVE_DATA`: ナレッジソースから生成的な変更を加えずに直接データを返す。既存の機能。

https://learn.microsoft.com/ja-jp/azure/search/search-agentic-retrieval-how-to-synthesize

### 3.3. Retrieval Instructions（取得手順）

複数のナレッジ ソースがある場合は、次のプロパティを設定して、クエリ計画を特定のナレッジソースに制限します。

- `always_query_source`: クエリの計画に常に当該ナレッジソースが含まれます。既定値は `False`
- `retrieval_instructions`: **回答にナレッジソースを含めたり除外したりする指示を自然言語で記述**します。例：「_質問に関する情報を検索し、適切な回答を提供してください。製品ついての問い合わせには、製品ナレッジソースのみを使用してください。それ以外の場合は、Wikipedia ナレッジソースをご利用ください。_」

**`always_query_source` プロパティは、`retrieval_instructions` を上書きします。** 取得手順を指定するときは、必ず `always_query_source` を `False` に設定する必要があります。

### 3.4. Include Activity, References, ReferenceSourceData
検索結果の分析、チューニングに使えるデバッグ情報は以下を設定可能です。

- クエリの実行と経過時間に関する分析情報を得るために、 `include_activity` を `True` (既定) に設定したままにします。（`KnowledgeAgentOutputConfiguration` クラス）

- 個別にスコア付けされた各結果の詳細については、 `include_references` 設定を `True` (既定) に保持します。（`KnowledgeSourceReference` クラス）

- インデックスの逐語的な内容が不要な場合は、 `include_reference_source_data` を `False` に設定します。 この情報を省略すると、応答が簡略化され、読みやすくなります。（`KnowledgeSourceReference` クラス）


### Get Knowledge Agent

In [ ]:
agent = search_client.get_agent(knowledge_agent_name)
agent.as_dict()

### List Knowledge Agent

In [ ]:
agent = search_client.list_agents()
for agent in agent:
    print(f"Knowledge agent '{agent.name}' ")


### Delete Knowledge Agent

In [ ]:
# response = search_client.delete_agent(knowledge_agent_name)
# print(response)

## Set up messages
メッセージは検索ルートの入力となり、会話履歴を含みます。各メッセージには、システムやユーザーなど発信元を示す役割と、自然言語によるコンテンツが含まれます。使用するLLMによって有効な役割が決まります。

In [ ]:
instructions = """
丁寧に質問に答えるQ&Aエージェント。
答えがわからない場合は「わかりません」と応答してください。
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

## エージェント型検索を用いて結果を取得する
このステップでは、エージェント型検索パイプラインを実行し、根拠に基づき引用付きで裏付けられた回答を生成します。会話履歴と検索パラメータに基づき、ナレッジエージェントは以下の処理を行います：

1. 会話全体を分析し、ユーザーの情報ニーズを推測します。
1. 複合クエリを焦点化されたサブクエリに分解します。
1. サブクエリを知識ソースに対して並行実行します。
1. セマンティックランカーを用いて結果を再ランク付け・フィルタリングします。
1. 上位結果を統合し、自然言語の回答を生成します。

In [ ]:

from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import (
    KnowledgeAgentRetrievalRequest,
    KnowledgeAgentMessage,
    KnowledgeAgentMessageTextContent,
    SearchIndexKnowledgeSourceParams,
)

print(f"Knowledge Agent: {knowledge_agent_name} から取得")

agent_client = KnowledgeAgentRetrievalClient(endpoint=SEARCH_ENDPOINT, agent_name=knowledge_agent_name, credential=AzureKeyCredential(SEARCH_KEY))
query_1 = """
平清盛と平維盛の生誕地とそれぞれの功績を教えて
"""

messages.append({
    "role": "user",
    "content": query_1
})

req = KnowledgeAgentRetrievalRequest(
    messages=[
        KnowledgeAgentMessage(
            role=m["role"],
            content=[KnowledgeAgentMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    # knowledge_source_params=[
    #     SearchIndexKnowledgeSourceParams(
    #         knowledge_source_name=knowledge_source_name,
    #         kind="searchIndex"
    #     )
    # ]
)

result = agent_client.retrieve(retrieval_request=req, api_version=search_api_version)
print(f"Retrieved content from '{knowledge_source_name}' successfully.")

In [ ]:
result.as_dict()

### 結果の分析
ナレッジエージェントは回答合成用に設定されているため、検索応答には以下の値が含まれます。

- `response_content`: 検索クエリに対する LLM 生成の回答（検索された文書を引用したもの）
- `activity_content`: 詳細な計画と実行情報（サブクエリ、再順位付けの決定、中間ステップを含む）
- `references_content`: 回答に貢献したソース文書とチャンク

ヒント：リランキングのしきい値やナレッジソースのプロパティといった検索パラメータは、エージェントがどの程度積極的に再ランク付けを行い、どの情報源を照会するかに影響します。アクティビティと参照情報を確認し、根拠の妥当性を検証するとともに、追跡可能な引用を構築してください。

In [ ]:
response_contents = []
activity_contents = []
references_contents = []

In [ ]:
import json

# response_content、activity_content、および references_content 用のシンプルな文字列値を構築する

# Responses -> text/value フィールドを連結
response_content = "\n\n".join([
    text for resp in getattr(result, "response", [])
    for content in getattr(resp, "content", [])
    if (text := getattr(content, "text", None) or getattr(content, "value", None) or str(content))
]) or "No response found on 'result'"

response_contents.append(response_content)

# Print the three string values
print("response_content:\n", response_content, "\n")

In [ ]:
# Activity -> JSON string of activity as list of dicts
activity_content = (
    json.dumps([a.as_dict() for a in result.activity], indent=2, ensure_ascii=False)
    if getattr(result, "activity", None) 
    else "No activity found on 'result'"
)
    
activity_contents.append(activity_content)
print("activity_content:\n", activity_content, "\n")

In [ ]:
# References -> JSON string of references as list of dicts
if getattr(result, "references", None):
    references_content = json.dumps([r.as_dict() for r in result.references], indent=2, ensure_ascii=False)
else:
    references_content = "No references found on 'result'"
    
references_contents.append(references_content)
print("references_content:\n", references_content)

# Acknowledgment

https://github.com/Azure-Samples/azure-search-python-samples/blob/main/Quickstart-Agentic-Retrieval/quickstart-agentic-retrieval.ipynb